# Job assignment problem solving notebook

This notebook runs through modelling and then solving a problem instance for an modified assignment problem based on our work on airspace sectorisation and ATC staff allocation.

Another notebook covers the problem generation in more detail.

Documentation for a specific version of the problem we worked on for Air Services Australia can be found at:

https://confluence.mclaren.com/display/DITMPDI/1.6.1.2.+Aviation+-+ASA

The key features of the problem are:

- A time horizon - a set of discrete time intervals representing a period of time

- Set of jobs - possibly distinct by their taskload over the time horizon time

- Job taskload - a measure of work associated with each job in each time interval

- Set of agents/workers - possibly distinct by their capability to work on certain jobs and available times

- A grouping of jobs - jobs grouped together to be performed by one agent simultaniously

The objectives are to:

- Minimise agent manhours - minimise ATC time on consoles

- Minimise job and agent movement between job groups - this is done to try and smooth the solutions and stop so many changeovers

In [1]:
import numpy as np
import numpy.polynomial.polynomial as poly
import pandas as pd
import os
import copy
import warnings
import datetime
from time import ctime
import scipy.stats as sc
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.core.base import Constraint as pyo_constraint
from pyomo.core.base import Var as pyo_vars

from generate_ASA_test_problem_functions import *


## Model the problem

We're going to model the problem as a MILP, formulated as:


__Sets__

$\mathcal{J}=\{j\}$: Set of jobs

$\mathcal{A}=\{a\}$: Set of agents

$\mathcal{G}=\{g\}$: Set of groups that jobs are assigned to

__Decision variables__

$p_{jgt} = \left\{ \begin{matrix}1 \text{ if job } j \text{ is assigned to group } g \text{ at time period } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$q_{agt} = \left\{ \begin{matrix}1 \text{ if agent } a \text{ is assigned to group } g \text{ at time period } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$r_{gt} = \left\{ \begin{matrix}1 \text{ if group } g \text{ is active in time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$s_{at} = \left\{ \begin{matrix}1 \text{ if agent } a \text{ is working in time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$w_{jgt} = \left\{ \begin{matrix}1 \text{ if job } j \text{ is moved to or from group } g \text{ at the beginning of time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$x_{gt} = \left\{ \begin{matrix}1 \text{ if any job moved to or from group } g \text{ at the beginning of time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$y_{agt} = \left\{ \begin{matrix}1 \text{ if agent } a \text{ is moved to or from group } g \text{ at the beginning of time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

__Parameters__

$f_{1}$: Weight factor applied to the number of times a group gains or loses a job at the beginning of a time interval

$f_{2}$: Weight factor applied to the number of jobs a group gains or loses at the beginning of a time interval

$f_{3}$: Weight factor applied to the number of times a group gains or loses an agent at the beginning of a time interval

$T_{max}$: The taskload limit - upper limit on taskload an agent(/group) can take on in any time interval

$L_{max}$: The maximum number of time intervals an agent can work consecutively

$L_{min}$: The minimum number of time intervals an agent must work consecutively

$M_{min}$: The minimum number of time intervals for an agents break between controlling

__Data__:

$T_{jt}$: The taskload for job $j$ in time interval $t$

$K_{at} = \left\{ \begin{matrix}1 \text{ if agent } a \text{ is on shift in time interval } t \\ 0 \text{ otherwise} \end{matrix} \right. $ 

__Objective function__

The first term minimises the number of groups being used over all time intervals. Since all jobs need to be assigned to a group and all groups with jobs need to be assigned an agent, this is equivelent to minimising the number of agent manhours over all time intervals.

The other terms are regularising terms to smooth the solution. The first two penalise jobs moving around groups too often, while the last one penalises agents moving between groups too often.

$$ \min \left( \sum_{g \in \mathcal{G}} \sum_{t \in \mathcal{T}} r_{gt} + f_{1} \sum_{t=1}^{\vert\mathcal{T}\vert-1} \sum_{g \in \mathcal{G}} x_{gt} + f_{2}  \sum_{t=1}^{\vert\mathcal{T}\vert-1} \sum_{j \in \mathcal{J}} \sum_{g \in \mathcal{G}} w_{jgt} + f_{3} \sum_{t=1}^{\vert\mathcal{T}\vert-1} \sum_{a \in \mathcal{A}} \sum_{g \in \mathcal{G}} y_{agt} \right) $$

__Constraints__

The following constraints are needed to link variable sets $p$, $q$, $r$, and $s$ together:

$$ \sum_{j \in \mathcal{J}} p_{jgt} \leq \vert\mathcal{J}\vert \sum_{a \in \mathcal{A}} q_{agt} \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T} $$

$$ \sum_{j \in \mathcal{J}} p_{jgt} \geq \sum_{a \in \mathcal{A}} q_{agt} \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T} $$

$$ \sum_{j \in \mathcal{J}} p_{jgt} \leq \vert\mathcal{J}\vert r_{gt} \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T} $$

$$ \sum_{j \in \mathcal{J}} p_{jgt} \geq  r_{gt} \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T} $$

$$ \sum_{a \in \mathcal{A}} q_{agt} =  s_{at} \hspace{0.5cm}  \forall a \in \mathcal{A}, \forall t \in \mathcal{T} $$

These constraints link variable sets $p$, $w$, and $x$ together:

$$ w_{jgt} \geq p_{jgt} - p_{jgt-1} \hspace{0.5cm} \forall j \in \mathcal{J}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ w_{jgt} \geq p_{jgt-1} - p_{jgt} \hspace{0.5cm} \forall j \in \mathcal{J}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ w_{jgt} \leq p_{jgt-1} + p_{jgt} \hspace{0.5cm} \forall j \in \mathcal{J}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ w_{jgt} \leq 2 - p_{jgt-1} - p_{jgt} \hspace{0.5cm} \forall j \in \mathcal{J}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ \sum_{j \in \mathcal{J}} w_{jgt} \leq \vert\mathcal{J}\vert x_{gt} \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T} $$

These constraints link variable sets $q$ and $y$ together:

$$ y_{agt} \geq q_{agt} - q_{agt-1} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ y_{agt} \geq q_{agt-1} - q_{agt} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ y_{agt} \leq q_{agt-1} + q_{agt} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

$$ y_{agt} \leq 2 - q_{agt-1} - q_{agt} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall g \in \mathcal{G}, \forall t \in \left[1, \vert \mathcal{T} \vert -1 \right] $$

This constraint ensures that every job is assigned a group in all time intervals:

$$\sum_{g \in \mathcal{G}} p_{jgt} = 1 \hspace{0.5cm} \forall j \in \mathcal{J}, \forall t \in \mathcal{T}$$

This constraint ensures that there is at max one agent per group in any time interval:

$$\sum_{a \in \mathcal{A}} q_{agt} \leq 1 \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T}$$

This constraint ensures that each agent is assigned no more than one group in any time interval:

$$\sum_{g \in \mathcal{G}} q_{agt} \leq 1 \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \mathcal{T}$$

This constraint ensures that no group is given a taskload it can't handle in any time interval. Due to the one to one mapping between agents and groups, this is equivilent to limiting agent taskload:

$$\sum_{j \in \mathcal{J}} (T_{jt} \times p_{jgt}) \leq T_{max} \hspace{0.5cm} \forall g \in \mathcal{G}, \forall t \in \mathcal{T}$$

This constraint limits the agents to only working within their shift hours:

$$s_{at} \leq K_{at} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \mathcal{T}$$

This constraint sets the maximum number of time intervals an agent can work consecutively:

$$ L_{max} - \sum_{i = t}^{t+L_{max}-1} s_{ai} \geq s_{a, t+A} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \left[0, \vert \mathcal{T} \vert - L_{max} - 1 \right] $$

These constraints set the minimum number of time intervals an agent must work consecutively:

$$L_{min}\left(s_{a,t-1} + \left(1-s_{at}\right)\right) \geq L_{min} - 1 - \sum_{i=t+1}^{t+L_{min}-1} s_{ai} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \left[1, \vert \mathcal{T} \vert - L_{min} \right] $$

$$(L_{min}-1) s_{a,0} \leq \sum_{i=1}^{L_{min}-1} s_{ai}$$

$$(L_{min}-1) s_{a,\vert \mathcal{T} \vert - L_{min}} \geq \sum_{i=\vert \mathcal{T} \vert-L_{min}+1}^{\vert \mathcal{T} \vert-1} s_{ai}$$

This constraint sets the minimum break time an agent can have:

$$M_{min}\left(s_{a,t} + \left(1-s_{at-1}\right)\right) \geq \sum_{i=t+1}^{t+M_{min}-1} s_{ai} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \left[1, \vert \mathcal{T} \vert - M_{min} \right] $$

This constraint breaks the symmetry between groups. Since all the job groups are identical, we can impose a precedence on them that discounts symmetric solutions, significantly helping solve time:

$$r_{gt} \geq r_{g+1t} \hspace{0.5cm} \forall g \in \left[0, \vert \mathcal{G} \vert - 2 \right], \forall t \in \mathcal{T} $$

Define some helper functions:

In [2]:
def taskload_array_to_dict(taskload_array):
    
    out_dict = {}
    
    for i in range(taskload_array.shape[0]):
        for j in range(taskload_array.shape[1]):
            out_dict[(i, j)] = taskload_array[i, j]
            
    return out_dict

def agent_availability_dict_processer(taskload_parameters, agent_availability_time_intervals_dict):
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    out_dict = {}
    
    for k, v in agent_availability_time_intervals_dict.items():
        for ti in range(number_time_intervals):
            if v[0] <= ti <= v[1]:
                out_dict[(k, ti)] = 1
            else:
                out_dict[(k, ti)] = 0
                
    return out_dict

Define potential objectives and constraint functions:

In [3]:
def objective_function_minimise_taskload(model):
    
    return sum(sum(model.r[ g, t]  for g in model.G) for t in model.T)

def objective_function_minimise_taskload_and_group_reconfigurations(model):
    
    return sum(sum(model.r[ g, t]  for g in model.G) for t in model.T) + \
           model.group_any_reconfig_weight * sum(sum(model.x[g, t] for g in model.G) for t in model.T_minus) + \
           model.group_job_reconfig_weight * sum(sum(sum(model.w[j, g, t] for j in model.J) for g in model.G) \
            for t in model.T_minus)

def objective_function_minimise_taskload_and_group_and_agent_reconfigurations(model):
    
    return sum(sum(model.r[ g, t]  for g in model.G) for t in model.T) + \
           model.group_any_reconfig_weight * sum(sum(model.x[g, t] for g in model.G) for t in model.T_minus) + \
           model.group_job_reconfig_weight * sum(sum(sum(model.w[j, g, t] for j in model.J) for g in model.G) \
                                             for t in model.T_minus) + \
           model.group_agent_reconfig_weight * sum(sum(sum(model.y[a, g, t] for a in model.A) for g in model.G) \
                                               for t in model.T_minus)

In [4]:
def constraint_link_p_and_q_variables(model, g, t):
    
    return sum(model.p[j, g, t] for j in model.J) <= model.number_jobs * sum(model.q[a, g, t] for a in model.A)

def constraint_link_p_and_q_variables_2(model, g, t):
    
    return sum(model.p[j, g, t] for j in model.J) >= sum(model.q[a, g, t] for a in model.A)

def constraint_link_p_and_r_variables(model, g, t):
    
    return sum(model.p[j, g, t] for j in model.J) <= model.number_jobs * model.r[g, t]

def constraint_link_p_and_r_variables_2(model, g, t):
    
    return sum(model.p[j, g, t] for j in model.J) >= model.r[g, t]

def constraint_link_q_and_s_variables(model, a, t):
    
    return sum(model.q[a, g, t] for g in model.G) == model.s[a, t] # model.number_groups * model.s[a, t]

def constraint_total_coverage(model, j, t):
    
    return sum(model.p[j, g, t] for g in model.G) == 1

def constraint_one_agent_per_group(model, g, t):
    
    return sum(model.q[a, g, t] for a in model.A) <= 1

def constraint_one_group_per_agent(model, a, t):
    
    return sum(model.q[a, g, t] for g in model.G) <= 1

def constraint_max_tasks(model, g, t):
    
    return sum(taskload_array[j, t] * model.p[j, g ,t] for j in model.J) <= model.taskload_limit

def constraint_limit_agent_working_hours(model, a, t):

    return model.s[a, t] <= model.agent_availability[a, t]

def constraint_max_working_time(model, a, t):
    
    return (model.max_working_time - sum(model.s[a, t2] for t2 in range(t, t + model.max_working_time)) >= 
           model.s[a, t + model.max_working_time])

def constraint_min_working_time(model, a, t):
    
    if t == 0:
        return ((model.min_working_time-1) * model.s[a, t] <= 
                sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_working_time)))
    if t == model.number_time_intervals - model.min_working_time:
        return ((model.min_working_time-1) * model.s[a, t] >= 
                sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_working_time)))
    else:
        return (model.min_working_time * (model.s[a, t - 1] + (1-model.s[a, t])) >= 
                (model.min_working_time - 1 - sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_working_time))))
    
def constraint_min_break_time(model, a, t):

    return (model.min_break_time * ((1 - model.s[a, t - 1]) + model.s[a, t]) >= 
            sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_break_time)))   

def constraint_break_symmetry(model, g, t):
    
    return model.r[g, t] >= model.r[g + 1, t] 

def constraint_link_p_and_w_variables_1(model, j, g, t):
    
    return model.w[j, g, t] >= model.p[j, g, t] - model.p[j, g, t-1] 

def constraint_link_p_and_w_variables_2(model, j, g, t):
    
    return model.w[j, g, t] >= model.p[j, g, t-1] - model.p[j, g, t] 

def constraint_link_p_and_w_variables_3(model, j, g, t):
    
    return model.w[j, g, t] <= model.p[j, g, t] + model.p[j, g, t-1] 

def constraint_link_p_and_w_variables_4(model, j, g, t):
    
    return model.w[j, g, t] <= 2 - model.p[j, g, t] - model.p[j, g, t-1] 

def constraint_link_w_and_x_variables(model, g, t):
    
    return sum(model.w[j, g, t] for j in model.J) <= model.number_jobs * model.x[g, t]

def constraint_link_q_and_y_variables_1(model, a, g, t):
    
    return model.y[a, g, t] >= model.q[a, g, t] - model.q[a, g, t-1] 

def constraint_link_q_and_y_variables_2(model, a, g, t):
    
    return model.y[a, g, t] >= model.q[a, g, t-1] - model.q[a, g, t] 

def constraint_link_q_and_y_variables_3(model, a, g, t):
    
    return model.y[a, g, t] <= model.q[a, g, t] + model.q[a, g, t-1] 

def constraint_link_q_and_y_variables_4(model, a, g, t):
    
    return model.y[a, g, t] <= 2 - model.q[a, g, t] - model.q[a, g, t-1] 

Define the model:

In [5]:
def build_model(taskload_parameters, agent_parameters, taskload_array, number_groups,
                group_any_reconfig_weight, group_job_reconfig_weight,
                group_agent_reconfig_weight):
    
    agent_availability_time_intervals_dict = agent_parameters['agent_availability_time_intervals_dict']
    max_working_time = agent_parameters['max_working_time']
    min_working_time = agent_parameters['min_working_time']
    min_break_time = agent_parameters['min_break_time']
    number_agents = agent_parameters['number_agents']
    taskload_limit = agent_parameters['taskload_limit']
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    # define model
    model = pyo.ConcreteModel()
    
    # define parameters
    model.taskload_limit = pyo.Param(initialize=taskload_limit, 
                                     within=pyo.NonNegativeReals)
    model.number_time_intervals = pyo.Param(initialize=number_time_intervals, 
                                            within=pyo.NonNegativeReals)
    model.number_agents = pyo.Param(initialize=number_agents, 
                                         within=pyo.NonNegativeReals)
    model.number_jobs = pyo.Param(initialize=number_jobs, 
                                  within=pyo.NonNegativeReals)
    model.number_groups = pyo.Param(initialize=number_groups, 
                                  within=pyo.NonNegativeReals)
    model.max_working_time = pyo.Param(initialize=max_working_time, 
                                       within=pyo.NonNegativeReals)
    model.min_working_time = pyo.Param(initialize=min_working_time, 
                                       within=pyo.NonNegativeReals)
    model.min_break_time = pyo.Param(initialize=min_break_time, 
                                       within=pyo.NonNegativeReals)
    model.group_any_reconfig_weight = pyo.Param(initialize=group_any_reconfig_weight, 
                                                within=pyo.NonNegativeReals)
    model.group_job_reconfig_weight = pyo.Param(initialize=group_job_reconfig_weight, 
                                                within=pyo.NonNegativeReals)
    model.group_agent_reconfig_weight = pyo.Param(initialize=group_agent_reconfig_weight, 
                                                within=pyo.NonNegativeReals)
    
    # define model sets
    model.T = pyo.RangeSet(0, model.number_time_intervals-1, 1, dimen=1, ordered=True)
    model.A = pyo.RangeSet(0, model.number_agents-1, dimen=1)
    model.J = pyo.RangeSet(0, model.number_jobs-1, dimen=1)
    model.G = pyo.RangeSet(0, model.number_groups-1, dimen=1)
    model.G_minus = pyo.RangeSet(0, model.number_groups-2, dimen=1)
    model.T_minus = pyo.RangeSet(1, model.number_time_intervals-1, 1, dimen=1, ordered=True)
    model.T_MaxWork = pyo.RangeSet(0, model.number_time_intervals - model.max_working_time-1, 
                                   1, dimen=1, ordered=True)
    model.T_MinWork = pyo.RangeSet(0, model.number_time_intervals - model.min_working_time, 
                                   1, dimen=1, ordered=True)
    model.T_MinBreak = pyo.RangeSet(1, model.number_time_intervals - model.min_break_time, 1, dimen=1, ordered=True)
    
    # add data to model
    model.taskload = pyo.Param(model.J, model.T, within=pyo.NonNegativeReals, 
                               initialize=taskload_array_to_dict(taskload_array), default=0)
    model.agent_availability = pyo.Param(model.A, model.T, within=pyo.NonNegativeReals, 
                                         initialize=agent_availability_dict_processer(taskload_parameters, 
                                                          agent_availability_time_intervals_dict), 
                                         default=0)
    
    # define decision variables
    model.p = pyo.Var(model.J, model.G, model.T, within=pyo.Binary)
    model.q = pyo.Var(model.A, model.G, model.T, within=pyo.Binary)
    model.r = pyo.Var(model.G, model.T, within=pyo.Binary)
    model.s = pyo.Var(model.A, model.T, within=pyo.Binary)
    
    model.w = pyo.Var(model.J, model.G, model.T_minus, within=pyo.Binary)
    model.x = pyo.Var(model.G, model.T_minus, within=pyo.Binary)
    
    model.y = pyo.Var(model.A, model.G, model.T_minus, within=pyo.Binary)
    
    # define objective function and constraints
    #model.objective_function_minimise_taskload = pyo.Objective(rule=objective_function_minimise_taskload,
    #                                                          sense=pyo.minimize)
    
    #model.objective_function_minimise_taskload_and_group_reconfigurations \
    #= pyo.Objective(rule=objective_function_minimise_taskload_and_group_reconfigurations,sense=pyo.minimize)
    
    model.objective_function_minimise_taskload_and_group_and_agent_reconfigurations \
    = pyo.Objective(rule=objective_function_minimise_taskload_and_group_and_agent_reconfigurations,sense=pyo.minimize)
    
    model.constraint_link_p_and_q_variables = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_q_variables)
    model.constraint_link_p_and_q_variables_2 = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_q_variables_2)
    model.constraint_link_p_and_r_variables = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_r_variables)
    model.constraint_link_p_and_r_variables_2 = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_r_variables_2)
    model.constraint_link_q_and_s_variables = pyo.Constraint(model.A, model.T, 
                                                              rule=constraint_link_q_and_s_variables)
    model.constraint_total_coverage = pyo.Constraint(model.J, model.T, 
                                                              rule=constraint_total_coverage)
    model.constraint_one_agent_per_group = pyo.Constraint(model.G, model.T, 
                                                            rule=constraint_one_agent_per_group)
    model.constraint_one_group_per_agent = pyo.Constraint(model.A, model.T, 
                                                            rule=constraint_one_group_per_agent)
    model.constraint_max_tasks = pyo.Constraint(model.G, model.T, 
                                                rule=constraint_max_tasks)
    model.constraint_limit_agent_working_hours = pyo.Constraint(model.A, model.T, 
                                                                rule=constraint_limit_agent_working_hours)
    model.constraint_max_working_time = pyo.Constraint(model.A, model.T_MaxWork, 
                                                       rule=constraint_max_working_time)
    model.constraint_min_working_time = pyo.Constraint(model.A, model.T_MinWork, 
                                                       rule=constraint_min_working_time)
    model.constraint_min_break_time = pyo.Constraint(model.A, model.T_MinBreak, 
                                                     rule=constraint_min_break_time)
    model.constraint_break_symmetry = pyo.Constraint(model.G_minus, model.T, 
                                                rule=constraint_break_symmetry)
    
    model.constraint_link_p_and_w_variables_1 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_1)
    model.constraint_link_p_and_w_variables_2 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_2)
    model.constraint_link_p_and_w_variables_3 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_3)
    model.constraint_link_p_and_w_variables_4 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_4)
    model.constraint_link_w_and_x_variables = pyo.Constraint(model.G, model.T_minus, 
                                                               rule=constraint_link_w_and_x_variables)
    
    model.constraint_link_q_and_y_variables_1 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_1)
    model.constraint_link_q_and_y_variables_2 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_2)
    model.constraint_link_q_and_y_variables_3 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_3)
    model.constraint_link_q_and_y_variables_4 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_4)
    
    return model

## Generate Problem

In this section we will generate some data for a sample problem, using functions from a previous notebook.

We first generate some taskload parameters. Here we are using: 

- 56 time intervals

- 8 jobs

- Job taskload (Normal) distribution means generated by N~(100, 10), with their standard deviations a quarter of the mean

- A low level of zero valued taskloads

- Taskloads distributed to be higher at the beginning and end of the time horizon

In [8]:
number_time_intervals = 56
number_jobs = 12

taskload_parameters = generate_random_taskload_parameters(number_time_intervals, number_jobs, 
                                                          {'taskload_dist_means_mean' : 100, 
                                                           'taskload_dist_means_stdev' : 10,
                                                           'taskload_dist_stdev_ratio' : 0.25},
                                                          zero_taskload_density=0.25, 
                                                          time_horizon_profile='middle_heavy', 
                                                          random_state=1)
taskload_parameters

{'number_time_intervals': 56,
 'number_jobs': 12,
 'job_normal_dist_params_dict': {0: {'mean': 116.0, 'stdev': 29.0},
  1: {'mean': 94.0, 'stdev': 24.0},
  2: {'mean': 95.0, 'stdev': 24.0},
  3: {'mean': 89.0, 'stdev': 22.0},
  4: {'mean': 109.0, 'stdev': 27.0},
  5: {'mean': 77.0, 'stdev': 19.0},
  6: {'mean': 117.0, 'stdev': 29.0},
  7: {'mean': 92.0, 'stdev': 23.0},
  8: {'mean': 103.0, 'stdev': 26.0},
  9: {'mean': 98.0, 'stdev': 24.0},
  10: {'mean': 115.0, 'stdev': 29.0},
  11: {'mean': 79.0, 'stdev': 20.0}},
 'job_time_horizon_profiles_dict': {0: 'middle_heavy',
  1: 'middle_heavy',
  2: 'middle_heavy',
  3: 'middle_heavy',
  4: 'middle_heavy',
  5: 'middle_heavy',
  6: 'middle_heavy',
  7: 'middle_heavy',
  8: 'middle_heavy',
  9: 'middle_heavy',
  10: 'middle_heavy',
  11: 'middle_heavy'},
 'job_zero_taskload_probability_dict': {0: 0.028291534783718025,
  1: 0.03312486445189117,
  2: 0.03286300675107521,
  3: 0.03451099613079475,
  4: 0.029643544174272844,
  5: 0.0384708654242

We then generate a taskload array from these parameters, where lighter colours indicate higher taskloads:

In [9]:
taskload_array = generate_taskload_array(taskload_parameters, random_state=0)
create_taskload_heatmap(taskload_parameters, taskload_array, in_notebook=True, 
                        colorscale='Greens',annotate_heatmap=False)

We then generate some agent parameters. Here we are using:

- 12 agents

- Start time ranging from time interval 0 to 24

- Shifts with 32 time intervals

- A maximum consecutive job working time of 8 time intervals

- A minimum consecutive job working time of 3 time intervals

- A minimum break time of 2 time intervals

In [10]:
number_jobs = taskload_parameters['number_jobs']

agent_parameters = {
    'number_agents' : 12
}

agent_availability_time_intervals_dict = {
    0 : (0, 31),
    1 : (0, 31),
    2 : (0, 31),
    3 : (8, 39),
    4 : (8, 39),
    5 : (8, 39),
    6 : (16, 47),
    7 : (16, 47),
    8 : (16, 47),
    9 : (24, 55),
    10 : (24, 55),
    11 : (24, 55)
}

agent_parameters['agent_availability_time_intervals_dict'] = agent_availability_time_intervals_dict
agent_parameters['taskload_limit'] = 600
agent_parameters['max_working_time'] = 8
agent_parameters['min_working_time'] = 3
agent_parameters['min_break_time'] = 2
taskload_limit = agent_parameters['taskload_limit']
agent_parameters

{'number_agents': 12,
 'agent_availability_time_intervals_dict': {0: (0, 31),
  1: (0, 31),
  2: (0, 31),
  3: (8, 39),
  4: (8, 39),
  5: (8, 39),
  6: (16, 47),
  7: (16, 47),
  8: (16, 47),
  9: (24, 55),
  10: (24, 55),
  11: (24, 55)},
 'taskload_limit': 600,
 'max_working_time': 8,
 'min_working_time': 3,
 'min_break_time': 2}

The heatmap below shows the availability of our agents over the time horizon:

In [11]:
number_agents = agent_parameters['number_agents']
number_time_intervals = taskload_parameters['number_time_intervals']
a = generate_agent_availability_array(number_agents, number_time_intervals, agent_availability_time_intervals_dict)

trace = go.Figure(data=[go.Heatmap(z=np.invert(a.astype(int))
                , x=[i for i in range(number_time_intervals)]
                , y=[i for i in range(number_agents)]
                , xgap=1, ygap=1
                , colorscale='Greens'
                , showscale=False
                )])
trace.layout.update(title = 'Working Availability',
                    xaxis = go.layout.XAxis(title='Time Interval'),
                    yaxis = go.layout.YAxis(title='Worker ID'))
iplot(trace)

## Solve model

In this section we will solve the formulated model.

In [12]:
def solve_instance(model, solver, solver_filepath, max_run_time=600, ratio_gap=0.1
                   , tee=False, warmstart=False, logfile_name = 'logfile.txt'):
    
    print('Run start time: ' + str(ctime()))

    # Use solvers on neos server
    if solver_filepath == 'neos':
        solver_manager = pyo.SolverManagerFactory('neos')
        opt = SolverFactory(solver)
        if solver == 'cplex':
            opt.set_options('mipgap=' + str(ratio_gap))
            opt.set_options('timelimit=' + str(max_run_time))
            opt.set_options('mipdisplay=' + str(3))
            opt.set_options('nodefile=' + str(2))
            opt.set_options('treememory=' + str(10000))
        elif solver == 'mosek':
            opt.set_options('MSK_DPAR_MIO_REL_GAP_CONST=' + str(ratio_gap))
            opt.set_options('MSK_DPAR_OPTIMIZER_MAX_TIME=' + str(max_run_time))
        results = solver_manager.solve(model, opt=opt, keepfiles=True, tee=tee)

    # Use local solver
    else:
        if solver == 'glpk':
            opt = SolverFactory(solver, executable=solver_filepath)
            opt.set_options('tmlim=' + str(max_run_time))
            results = opt.solve(model, tee=tee)
        elif solver == 'cbc':
            opt = SolverFactory(solver, executable=solver_filepath)
            opt.set_options('sec=' + str(max_run_time))
            opt.set_options('ratioGap=' + str(ratio_gap))
            if logfile_name == '':
                results = opt.solve(model, tee=tee, warmstart=warmstart)
            else:
                results = opt.solve(model, logfile=logfile_name, tee=tee, warmstart=warmstart)
        else:
            raise ValueError(f'Solver {solver} not supported')
            
    print('Run finish time: ' + str(ctime()))        
    
    return model, results

Here we build then solve the model. We have to supply a few input parameters to the model:

- The number of groups to model - this can be worked out by finding the maximum number of groups needed for a single time interval in the time horizon. While this is often easy to work out by inspection, in a production system there would likely need to be some kind of heuristic to work this out.

- The penalty attached to a group if a job is moved from/to it. This helps to smooth the solution against erratic job movement.

- This is a similar penalty as the one above, except it is at the job level, rather than the group level. It too helps with solution smoothing.

- The penalty attached to an agent if they move on/off a job. This helps with solution smoothing from an agent perspective.

In [13]:
number_groups = 3
group_any_reconfig_weight = 1.5
group_job_reconfig_weight = 0.025
group_agent_reconfig_weight = 0.1

m = build_model(taskload_parameters, agent_parameters, taskload_array, number_groups, 
                group_any_reconfig_weight,group_job_reconfig_weight, group_agent_reconfig_weight)

We can solve the model using either CPLEX on the NEOS server, or locally with CBC. We can specify the max solve time along with the ratio gap. In this instance we will use CPLEX:

In [ ]:
m, r = solve_instance(m, 'cplex', 'neos', max_run_time=2400, ratio_gap=0.01, tee=True)

Run start time: Fri Aug 30 11:00:15 2019


In [61]:
#m2, r2 = solve_instance(m, 'cbc', 'C:\\repos\\mat.core.decisioninsight.usecases\\solvers\\cbc\\bin\\cbc.exe', 
#                      max_run_time=300, ratio_gap=0.01, tee=True)

## Visualise/analyse the solution

In [13]:
def is_solution_valid(taskload_array, solution, partition_max):

    max_partition_index = int(solution.max())
    
    valid = True
    for ti in range(solution.shape[1]): 
        
        for partition in range(max_partition_index+1):
            
            job_indexes = np.where(solution[:,ti]==partition)[0]
            temp_array = np.zeros(solution.shape[0])
            temp_array[job_indexes] = 1
            job_workload = np.multiply(temp_array, taskload_array[:,ti]).sum()
                                                 
            if job_workload > partition_max:
                warnings.warn('solution not valid for partition ' + str(partition) + ' in time interval ' + str(ti))
                valid = False
        
    return valid

def make_solution_groups_array(model, taskload_parameters, agent_parameters, taskload_array):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    solution_array = np.zeros((number_jobs, number_time_intervals))
    
    
    
    job_time_to_group_dict = {(k2[0], k2[2]): k2[1] for k2, v2 in 
                                    {k: v for k, v in m.p.get_values().items() if v == 1
                                }.items()}
    
    for k, v in job_time_to_group_dict.items():
        
        solution_array[k[0],k[1]] = v
        
    return solution_array

def make_solution_agents_array(model, taskload_parameters, agent_parameters, taskload_array):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    solution_array = np.zeros((number_jobs, number_time_intervals))
    
    job_time_to_group_dict = {(k2[0], k2[2]): k2[1] for k2, v2 in 
                                    {k: v for k, v in m.p.get_values().items() if v == 1
                                }.items()}
    group_time_to_agent_dict = {(k2[1], k2[2]): k2[0] for k2, v2 in 
                                        {k: v for k, v in m.q.get_values().items() if v == 1
                                    }.items()}
    job_time_to_agent_dict = {k: group_time_to_agent_dict[(v, k[1])]  for k, v in job_time_to_group_dict.items()}
    
    for k, v in job_time_to_agent_dict.items():
        
        solution_array[k[0],k[1]] = v
        
    return solution_array

def make_solution_heatmap(model, taskload_parameters, agent_parameters, taskload_array, 
                          agent_heatmap = True, in_notebook=False, colorscale='Jet',
                          annotate_heatmap=True):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    if agent_heatmap:
        solution_array = make_solution_agents_array(model, taskload_parameters, agent_parameters, taskload_array)
    else:
        solution_array = make_solution_groups_array(model, taskload_parameters, agent_parameters, taskload_array)
    
    time_interval_taskloads = taskload_array.sum(axis=0)
    
    z_hover = np.empty((number_jobs, number_time_intervals),dtype='object')
    for i in range(number_time_intervals):
        for j in range(number_jobs):
            z_hover[j, i] = 'time interval ' + str(i) + '  job #' + str(j)
            
    if annotate_heatmap:
        annotation_text = taskload_array
    else:
        annotation_text = np.empty(taskload_array.shape,dtype='str')
        
    trace = ff.create_annotated_heatmap(z=solution_array
        , x=[i for i in range(number_time_intervals)]
        , y=[i for i in range(number_jobs)]
        , xgap=1, ygap=1
        , annotation_text=annotation_text
        , colorscale=colorscale
        , text=z_hover
        , hoverinfo='text'
        , showscale=True
    )
    
    trace.layout.update({'title':'Job to Console Solution',
                        'xaxis':go.layout.XAxis(#title='Time Intervals',
                        tickvals = [i for i in range(number_time_intervals)],
                        ticktext = [str(int(i)) for i in time_interval_taskloads],
                        tickfont = {'size':10},
                        tickangle=30,
                        side='top'),                  
                                                
                         
                         'yaxis':go.layout.YAxis(title='Job ID')})
    
    if in_notebook:
        iplot(trace)
    else:
        plot(trace)

Here we can visualise the solution output. We can choose to look at it at a group level, or at an agent level. At a group level we have:

In [14]:
make_solution_heatmap(m, taskload_parameters, agent_parameters, taskload_array, 
                      agent_heatmap = False, in_notebook=True, annotate_heatmap=False)

We can see that the first few time periods only require one group, then we switch to two until late in the time horizon when we can switch back to one. It should be noted that there are time intervals in which we could have just one group but the optimiser chooses not to so as to keep the solution smoother.

Below is the heatmap, now at the agent level:

In [15]:
make_solution_heatmap(m, taskload_parameters, agent_parameters, taskload_array, 
                      agent_heatmap = True, in_notebook=True, annotate_heatmap=False)

We can see that the agents are assigned to just one of the consoles, and usually work the maximum allowed time of 8 time intervals. The solution is clearly quite smooth.

We might also want to get some stats from the solution.

In [16]:
def get_solution_stats(group_solution_array, agent_solution_array):
    
    solution_stats = {}
    
    # helper function to get number of unique elements in each column of an array
    def nunique_percol_sort(a):
        b = np.sort(a,axis=0)
        return (b[1:] != b[:-1]).sum(axis=0)+1
    
    solution_stats['time_intervals_working'] = nunique_percol_sort(group_solution_array).sum()
    
    group_job_reconfig_count = 0
    group_reconfig_count = 0
    for i in range(group_solution_array.shape[0]):
        for j in range(group_solution_array.shape[1]-1):
            if i == 0:
                if not np.array_equal(group_solution_array[:,j], group_solution_array[:,j+1]):
                    group_reconfig_count += 1
            if group_solution_array[i,j] != group_solution_array[i,j+1]:
                group_job_reconfig_count += 1
    
    solution_stats['group_job_reconfig_count'] = group_job_reconfig_count
    solution_stats['group_reconfig_count'] = group_reconfig_count
    
    agent_reconfig_count = 0
    for j in range(agent_solution_array.shape[1]-1):
        if not np.array_equal(agent_solution_array[:,j], agent_solution_array[:,j+1]):
            agent_reconfig_count += min(len(np.unique(agent_solution_array[:,j])), 
                                        len(np.unique(agent_solution_array[:,j+1])))
            
    solution_stats['agent_reconfig_count'] = agent_reconfig_count
    
    return solution_stats

In [17]:
a = make_solution_groups_array(m, taskload_parameters, agent_parameters, taskload_array)
b = make_solution_agents_array(m, taskload_parameters, agent_parameters, taskload_array)
get_solution_stats(a, b)

{'time_intervals_working': 108,
 'group_job_reconfig_count': 4,
 'group_reconfig_count': 1,
 'agent_reconfig_count': 21}

From the dictionary returned above we can see that there were 108 man-time-intervals used. On 1 occasions jobs were moved between groups, and in total 4 jobs were moved. There were 21 instances of controllers switching over.